In [14]:
''' NOT USED
import numpy as np
from bio_volumentations import Compose,\
    RandomGamma,\
    RandomRotate90,\
    GaussianBlur,\
    RandomAffineTransform,\
    RandomScale

# Create the transformation using Compose from a list of transformations
aug = Compose([
        RandomAffineTransform(p=1),
        RandomScale(p=1),
        RandomGamma(gamma_limit = (0.8, 1,2), p = 0.8),
        #RandomRotate90(axes = [1, 2, 3], p = 1),
        GaussianBlur(sigma = 1.2, p = 0.8),
      ],  p=1)

# Generate image and a corresponding labeled image
img = np.random.rand(1, 128, 256, 256)
lbl = np.random.randint(0, 1, size=(128, 256, 256), dtype=np.uint8)

# Transform the images
# Notice that the images must be passed as keyword arguments to the transformation pipeline
# and extracted from the outputted dictionary.
data = {'image': img, 'mask': lbl}
aug_data = aug(**data)
transformed_img, transformed_lbl = aug_data['image'], aug_data['mask']
'''

1. Define Transform

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2. Define sample & show sample

In [16]:
from bio_volumentations.napari_tools import show_sample
import numpy as np

def create_bbox(w, h, d, wd=5, hd=5, dd=5):
    return [
        [(w,      h, d), (w,      h, d + dd), (w,      h + hd, d + dd), (w,      h + hd, d)], # W
        [(w + wd, h, d), (w + wd, h, d + dd), (w + wd, h + hd, d + dd), (w + wd, h + hd, d)], # W+WD
        [(w, h     , d), (w, h     , d + dd), (w + wd, h     , d + dd), (w + wd, h     , d)], # H
        [(w, h + hd, d), (w, h + hd, d + dd), (w + wd, h + hd, d + dd), (w + wd, h + hd, d)], # H+HD
        [(w, h, d     ), (w, h + hd, d     ), (w + wd, h + hd, d     ), (w + wd, h, d     )], # D
        [(w, h, d + dd), (w, h + hd, d + dd), (w + wd, h + hd, d + dd), (w + wd, h, d + dd)], # D+DD
    ]

w, h, d = 60, 60, 60

img = np.random.rand(w, h, d)
mask = np.zeros((w, h, d), np.int32)
keypoints, bboxes = [], []

for i in range(15):
    w1, h1, d1 = np.random.randint(w-3), np.random.randint(h-3), np.random.randint(d-3)
    #img[:, w1, h1, d1] = 0
    mask[w1:w1+3, h1:h1+3, d1:d1+3] = i
    keypoints.append((w1+1., h1+1., d1+1.))
    bboxes += create_bbox(w1-1, h1-1, d1-1)

sample = {'image': img,
          'mask': mask,
          'keypoints': keypoints,
          #'bboxes': bboxes
          }

In [17]:
show_sample(sample)

(1, 60, 60, 60)


3. create and apply transform

In [19]:
from bio_volumentations import Scale, RandomScale, Compose, Resize, AffineTransform, RandomAffineTransform

params = {'scaling_limit': (.5, 2.)}  # RandomScale
params = {'shape': (60, 30, 120)}
params = {'angles': (0, 30, 0),
          'translation': (0, 0, 0),
          'scale': (1., 1.2, 1.3)} # AffineTransform

params = {'angle_limit': (45, 30, 30),
          'translation_limit': (5, 5, 5),
          'scaling_limit': (0.2, 0.1, 0.3)} # RandomAffineTransform

transform = RandomAffineTransform
tr = Compose([transform(**params, p=1)])
transformed_sample = tr(**sample)

4. show result

In [20]:
#transformed_sample['keypoints'] = transformed_sample['keypoints'] + sample['keypoints']
show_sample(transformed_sample)

(1, 60, 60, 60)


In [21]:
transformed_sample['keypoints']


[(45.31621988461572, 18.317907240203006, 47.35854337636425),
 (14.397169966506032, 10.454536431727412, 23.006883455868987),
 (36.81085646165554, 34.817981310331604, 35.518247652933454),
 (4.671621357519772, 44.23183077702797, 51.5048292834707),
 (40.31839311921126, 25.082711236109446, 30.272013399520123),
 (15.267496510456871, 19.88351086387614, 22.555933748070707),
 (24.837839154860607, 50.605495406588034, 12.126165356469556),
 (54.14022971153564, 17.156881233967376, 42.3573522663649),
 (24.700193136424986, 36.056193768664116, 20.50412318689227),
 (13.377920639269867, 10.268477626511395, 12.001457604855162),
 (24.16044638734531, 0.8720164715200056, 53.46845573269649),
 (1.7916278016560163, 37.40161317381529, 37.984617141936496),
 (13.823818556565376, 36.27349815175894, 41.14025990234516)]